In [2]:
import glob
import numpy as np
import pandas as pd
import numpy as np
from sglm.models import sglm
from sglm.features import gen_signal_df as gsd
from sglm.features import build_features as bf
from sglm.features import gen_signal_df as gsd
from sglm.features import build_features as bf

# Load Signal Data
signal_files = glob.glob(f'../../data/raw/GLM_SIGNALS_WT61_*')
table_files = [_.replace('GLM_SIGNALS', 'GLM_TABLE') for _ in signal_files]

files_list = signal_files
channel_definitions = {
        ('WT61',): {'Ch1': 'gACH', 'Ch2': 'rDA'},
        ('WT64',): {'Ch1': 'gACH', 'Ch2': 'empty'},
        ('WT63',): {'Ch1': 'gDA', 'Ch2': 'empty'},
    }

channel_assignments = bf.get_rename_columns_by_file(files_list, channel_definitions)

# Load Table Data
signal_fn = signal_files[0]
table_fn = table_files[0]

signal_df = pd.read_csv(signal_fn)
table_df = pd.read_csv(table_fn)
signal_filename_out = signal_fn.split('/')[-1].replace('GLM_SIGNALS', 'GLM_SIGNALS_INTERIM').replace('txt', 'csv')
table_filename_out = signal_fn.split('/')[-1].replace('GLM_TABLE', 'GLM_TABLE_INTERIM').replace('txt', 'csv')

%load_ext autoreload
%autoreload 2
signal_df, table_df = gsd.generate_signal_df(signal_fn,
                                        table_fn,
                                        signal_filename_out=f'../../data/interim/{signal_filename_out}',
                                        table_filename_out=f'../../data/interim/{table_filename_out}'
                                        )

('WT61',)
> GLM_SIGNALS_WT61_10152021.txt
> GLM_SIGNALS_WT61_10042021.txt
> GLM_SIGNALS_WT61_10062021.txt
> GLM_SIGNALS_WT61_10132021.txt
> GLM_SIGNALS_WT61_10082021.txt
> GLM_SIGNALS_WT61_10182021.txt
> GLM_SIGNALS_WT61_10112021.txt
('WT64',)
('WT63',)


NameError: name 'pd' is not defined

In [ ]:

# Break down Preprocess Lynne into component parts

# Rename Columns
df = bf.rename_consistent_columns(signal_df)
if signal_fn in channel_assignments:
    df = df.rename(channel_assignments[signal_file], axis=1)

## Set Reward Flags
df['r_trial'] = (df.groupby('nTrial')['photometrySideInIndexr'].transform(np.sum) > 0) * 1.0
df['nr_trial'] = (df.groupby('nTrial')['photometrySideInIndexnr'].transform(np.sum) > 0) * 1.0

## Define Side Rewarded / Unrewarded Flags
df = bf.set_port_entry_exit_rewarded_unrewarded_indicators(df)

## Define Side Agnostic Events
df = bf.define_side_agnostic_events(df)

# print('Percent of Data in ITI:', (df['nTrial'] == df['nEndTrial']).mean())
